In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google import genai

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", convert_system_message_to_human=True)

In [6]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate
from langchain_core.tools import tool


In [7]:
import json, re 

In [8]:
def _extract_str(maybe_json: str, key: str | None = None) -> str:
    """Accepts plain strings or JSON strings like '{"key": "value"}'.
       If key is provided, returns obj[key] when possible."""
    if isinstance(maybe_json, str):
        s = maybe_json.strip()
        # try to parse as JSON
        if s.startswith("{") and s.endswith("}"):
            try:
                obj = json.loads(s)
                if isinstance(obj, dict):
                    if key and key in obj and isinstance(obj[key], str):
                        return obj[key]
                    # if no key, fall back to the first string value
                    for v in obj.values():
                        if isinstance(v, str):
                            return v
            except Exception:
                pass
        return s
    # if somehow a dict slips through
    if isinstance(maybe_json, dict):
        if key and key in maybe_json and isinstance(maybe_json[key], str):
            return maybe_json[key]
        for v in maybe_json.values():
            if isinstance(v, str):
                return v
    return ""

In [9]:

@tool
def get_employee_id(name: str) -> str:
    """INPUT: employee name (case-insensitive). OUTPUT: ID like 'E005' or 'NOT_FOUND'."""
    name_to_id = {
        "alice":"E001","bob":"E002","charlie":"E003","diana":"E004","evan":"E005",
        "fiona":"E006","george":"E007","hannah":"E008","ian":"E009","jasmine":"E010",
    }
    name_str = _extract_str(name, key="name").lower()
    return name_to_id.get(name_str, "NOT_FOUND")

In [10]:
get_employee_id("Alice")

/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/ipykernel_30137/494535035.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_employee_id("Alice")


'E001'

In [11]:
@tool
def get_employee_salary(employee_id: str) -> str:
    """INPUT: employee ID like 'E003' (case-insensitive). OUTPUT: salary as text or 'INPUT_ERROR'/'NOT_FOUND'."""
    # Accept plain "E003" or JSON like {"employee_id":"E003"}
    raw = _extract_str(employee_id, key="employee_id").strip()
    # If the model mistakenly passed a name here, nudge it
    if not re.fullmatch(r"(?i)E\d{3}", raw):
        return "INPUT_ERROR"
    emp_id = raw.upper()

    salaries = {
        "E001":56000,"E002":47000,"E003":52000,"E004":61000,"E005":45000,
        "E006":58000,"E007":49000,"E008":53000,"E009":50000,"E010":55000
    }
    sal = salaries.get(emp_id)
    return "NOT_FOUND" if sal is None else str(sal)

In [12]:
get_employee_salary("E001")

'56000'

In [13]:
tools = [get_employee_id, get_employee_salary]

In [14]:
from langchain import hub
base = hub.pull("hwchase17/react").template

In [15]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    base + 
    "\nTool-use guidance:\n"
    "- If the user gives a NAME, first call get_employee_id(name). "
    "Then call get_employee_salary(employee_id) with the returned ID.\n"
    "- For get_employee_salary, pass only an ID like 'E003'.\n"
    "- After any Observation containing the requested value, produce Final Answer.\n"
    "After any Observation that contains the requested value, do NOT call more tools. \n"
)

In [16]:
print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} template="Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}\nTool-use guidance:\n- If the user gives a NAME, first call get_employee_id(name). Then call get_employee_salary(employee_id) with the returned ID.\n- For get_employee_salary, pass only an ID like 'E003'.\n- After any Observation containing the requested value, produce Final Answer.\nAft

In [17]:
agent = create_react_agent(llm, tools, prompt)

In [21]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,
                               handle_parsing_errors=True,
                               max_iterations=3, return_intermediate_steps=True)

In [22]:
agent_executor.invoke({"input": "What is Bob's salary?"})



> Entering new AgentExecutor chain...


/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: I need to find Bob's salary. First, I need to get his employee ID using the `get_employee_id` tool. Then, I can use the `get_employee_salary` tool with the obtained employee ID to find his salary.
Action: get_employee_id
Action Input: "Bob"E002

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: I have Bob's employee ID, which is E002. Now I need to use the `get_employee_salary` tool to find his salary.
Action: get_employee_salary
Action Input: "E002"47000

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: I have already found Bob's salary, which is 47000. I can now provide the final answer.
Final Answer: Bob's salary is 47000.

> Finished chain.


{'input': "What is Bob's salary?",
 'output': "Bob's salary is 47000.",
 'intermediate_steps': [(AgentAction(tool='get_employee_id', tool_input='Bob', log='Thought: I need to find Bob\'s salary. First, I need to get his employee ID using the `get_employee_id` tool. Then, I can use the `get_employee_salary` tool with the obtained employee ID to find his salary.\nAction: get_employee_id\nAction Input: "Bob"'),
   'E002'),
  (AgentAction(tool='get_employee_salary', tool_input='E002', log='Thought: I have Bob\'s employee ID, which is E002. Now I need to use the `get_employee_salary` tool to find his salary.\nAction: get_employee_salary\nAction Input: "E002"'),
   '47000')]}

In [23]:
agent_executor.invoke({"input": "What is the salary of employee with employee id E003?"})



> Entering new AgentExecutor chain...


/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: The user is asking for the salary of an employee with a given employee ID. I can use the `get_employee_salary` tool to find this information.
Action: get_employee_salary
Action Input: E00352000

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: The user is asking for the salary of an employee with a given employee ID. I have the tool `get_employee_salary` which takes an employee ID as input and returns the salary. I can directly use this tool to answer the question.
Action: get_employee_salary
Action Input: E00352000

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: The user is asking for the salary of an employee with a given employee ID, "E003". I have the tool `get_employee_salary` which takes an employee ID as input and returns the salary. I can directly use this tool to answer the question.
Action: get_employee_salary
Action Input: E00352000

> Finished chain.


{'input': 'What is the salary of employee with employee id E003?',
 'output': 'Agent stopped due to iteration limit or time limit.',
 'intermediate_steps': [(AgentAction(tool='get_employee_salary', tool_input='E003', log='Thought: The user is asking for the salary of an employee with a given employee ID. I can use the `get_employee_salary` tool to find this information.\nAction: get_employee_salary\nAction Input: E003'),
   '52000'),
  (AgentAction(tool='get_employee_salary', tool_input='E003', log='Thought: The user is asking for the salary of an employee with a given employee ID. I have the tool `get_employee_salary` which takes an employee ID as input and returns the salary. I can directly use this tool to answer the question.\nAction: get_employee_salary\nAction Input: E003'),
   '52000'),
  (AgentAction(tool='get_employee_salary', tool_input='E003', log='Thought: The user is asking for the salary of an employee with a given employee ID, "E003". I have the tool `get_employee_salary